In [1]:
cd /home/urwa/Documents/side_projects/urban/UrbanTrafficPrediction/

/home/urwa/Documents/side_projects/urban/UrbanTrafficPrediction


In [2]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [4]:
from utils.lstm_utils import prepare_data_lstm, lstm_monthly_dataloader
from utils.lstm_utils import get_device

from utils.lstm_utils import evaluate_edge_monthy
from models.models import LSTM

from utils.lstm_utils import train_one_epoch, store_chekpoint, evaluate_lstm_pipeline_model, run_inference

In [5]:
torch.manual_seed(2020)
np.random.seed(2020)

In [6]:
config = {'lr': 0.00034439316653688684,
 'layers': 3,
 'step_size': 11,
 'gamma': 0.761795969995615,
 'bptt': 19,
 'dropout': 0.1227497445640586}

In [7]:
device = get_device(cuda=True)
device

device(type='cuda')

In [8]:
data_path = '/home/urwa/Documents/side_projects/urban/data/featureData/com_jfk.csv'
weights_path = '/home/urwa/Documents/side_projects/urban/data/featureData/com_jfk_weights.csv'
test_data_path='/home/urwa/Documents/side_projects/urban/data/featureData/jfk.csv'

exp_dir = '/home/urwa/Documents/side_projects/urban/UrbanTrafficPrediction/data/lstm_12fold/jfk'

In [9]:
dataset, targetColumns, features_cols = prepare_data_lstm(data_path)

Raw Shape:  (8757, 113)
Cleaned Shape:  (8757, 38)
Target columns: 24
Feature coumns:  13


In [10]:
bptt = config['bptt']

R2List = []
EdgeR2List = []
residual_list = []


for m in range(1,13):
    
    print('-------------------------------------------------')
    print('-------------------------------------------------')
    print("Month: ", m)

    train_inout_seq, test_inout_seq = lstm_monthly_dataloader(dataset,features_cols, targetColumns, m,
                                                              bptt, device)
    
    lstm_layers = config['layers']
    network_size = len(targetColumns)
    feat_size = len(features_cols)
    dropout = config['dropout']
    hidden_layer_size=100
    
    
    model = LSTM(feat_size, network_size, hidden_layer_size, lstm_layers, dropout).to(device)
    loss_function = nn.L1Loss()   
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma']) 
    
    
    epochs = 2
    
    best_r2=0
    
    print("\n Training model...")
    for i in range(epochs):
        
        loss = train_one_epoch(model, optimizer, loss_function, train_inout_seq, device)
        scheduler.step()
        store_chekpoint(exp_dir, model, optimizer, scheduler)

        residual, r2, rmse, mae = evaluate_lstm_pipeline_model(model, test_inout_seq, device)
        print(f'epoch: {i:3} loss: {loss:10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

        if r2 > best_r2:
            best_r2 = r2
            best_residual = residual
            torch.save(model.state_dict(), os.path.join(exp_dir, str(m)+'.pt'))
            np.save(os.path.join(exp_dir,str(m)+'.npy'), best_residual)
            edge_res, edge_r2, edge_rmse, edge_mae = evaluate_edge_monthy(model, test_inout_seq, device, 
                                                                   targetColumns, weights_path, 
                                                              test_data_path, m, bptt)
            print('Saving model and residual')
            print('edge r2: ', edge_r2)
            
            
    print("best_r2: ", best_r2)

    R2List.append(best_r2)
    EdgeR2List.append(edge_r2)
    residual_list.append(best_residual)

-------------------------------------------------
-------------------------------------------------
Month:  1
train test split
train shape:  (8016, 38)
test shape:  (741, 38)
train feature tensor shape : torch.Size([8016, 13])
train target tensor shape : torch.Size([8016, 24])
test feature tensor shape : torch.Size([741, 13])
test target tensor shape : torch.Size([741, 24])

sequences
torch.Size([19, 13]) torch.Size([19, 24]) torch.Size([19, 24])

 Training model...
------- Saving checkpoint---------------
epoch:   0 loss: 8.91559761 r2: 0.376 rmse: 308.645 mae: 9.680
Saving model and residual
edge r2:  0.21851823390740718
------- Saving checkpoint---------------
epoch:   1 loss: 5.94415740 r2: 0.562 rmse: 216.613 mae: 7.795
Saving model and residual
edge r2:  0.34381119502283347
best_r2:  0.5617228564892284
-------------------------------------------------
-------------------------------------------------
Month:  2
train test split
train shape:  (8085, 38)
test shape:  (672, 38)
train

KeyboardInterrupt: 